## GeoCat experimental

In [ ]:
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)

CITY="madison"


In [ ]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})
df_checkin['date']=pd.to_datetime(df_checkin['date'])
# users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# num_users=len(users)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

In [ ]:
df_poi=pd.read_csv("../data/poi/"+CITY+".csv",converters={'categories':string_to_array})

# Test and training set

In [ ]:
df_checkin=df_checkin.sort_values(['user_id','date'])

ranks=df_checkin.groupby('user_id')['date'].rank(method='first').reset_index(drop=True)
ranks.head()

#ranks=df_review.groupby('user_id')['date'].rank(method='first')
count=df_checkin.reset_index()['user_id'].map(df_checkin.groupby('user_id')['date'].apply(len))
count.head()

training_set_delimiter=(ranks/count)<0.7
test_set_delimiter=np.logical_not(training_set_delimiter)
training_set_delimiter

df_checkin_train=df_checkin.reset_index()[training_set_delimiter]
df_checkin_train.count()

df_checkin_test=df_checkin.reset_index()[test_set_delimiter]
df_checkin_test.count()

users=df_checkin_train['user_id'].drop_duplicates().reset_index(drop=True)



In [ ]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [ ]:
num_users=len(users)
users.count()

In [ ]:
#df_checkin_train[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/train/"+CITY+".csv",index=False,header=False)

#df_checkin_test[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/test/"+CITY+".csv",index=False,header=False)

In [ ]:
df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_test=df_checkin_test.set_index('user_id')

### load poi neighbor

In [ ]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [ ]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)
dict_rec_list=dict()
for user_id in users:
    print(user_id)
    # User visits
    df_user_checkin=df_checkin_train.loc[[user_id]].reset_index(drop=True)
    
    list_user_pois=df_user_checkin['business_id'].drop_duplicates().values
    
    tmp_rec_list=rec.mostpopular(df_checkin_train.reset_index(),N,df_user_checkin,num_users,list_user_pois)
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi,on='business_id').drop_duplicates(subset=df_poi.columns.difference(['categories'])).reset_index(drop=True)

    #print(tmp_rec_list[['business_id','score']].head())

    business_cover=dict()
    for business_id in df_user_checkin['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.75 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
    relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)
    
    # post-processing for diversification
    for i in range(K):
#         start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            poi_neighbors=df_poi_neighbor.loc[poi_candidate.business_id].neighbors
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories_user,rec_list_categories,poi_neighbors)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate 
#         stop = timeit.default_timer()
#         print('Time:', stop - start)
        if poi_to_insert is not None:
            poi_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
    
    dict_rec_list[user_id]=rec_list[['business_id','score']]
    i+=1
    if i % num_users==0:
        print(i)

0   IMLrj2klosTFvPRLv56cng  0.260358
24  ujmEBvifdJM6h6RLv4wQIg  0.050891
68  V13IWgZvNps2Vo7WyEnD8A  0.024049
0   XXW_OFaYQkkGOGniujZFHg  0.000000
0   FvVSy2r7_zDEhZWqLgjXNQ  0.000000
0   WRQ-9LluyivReFiQZFUujw  0.000000
0   Vs7gc9EE3k9wARuUcN9piA  0.000000
0   Os1n1_idfw9vv9kwULGJnQ  0.000000
0   qsJkjQoM3Hdg6Gtqp2S9_w  0.000000
0   O7UMzd3i-Zk8dMeyY9ZwoA  0.000000
0   qmymSqVwHYRqdwfcBatzpQ  0.000000
0   pH0BLkL4cbxKzu471VZnuA  0.000000
0   78hGCrQ6wLA2yh8zMywnmw  0.000000
0   _pBXtjN43eqMV0XZTz7nmw  0.000000
0   tMYYSWyTppDcF070Hmy-kQ  0.000000
0   dm6sO_Y8JdKTE1ZM955yug  0.000000
0   Yiw2jUJzUdFmglHZgj4RaQ  0.000000
0   FhIeCF6QrsLaRvAeu0oEPQ  0.000000
0   5ma1OUQWy_Ds80xUiP7JPQ  0.000000
0   k2b3niokS_tosjah_rzCPw  0.000000